In [3]:
import json
import pickle
import tqdm
import pandas as pd
from conversation import create_coder, create_reviewer, create_refiner, start_conversation

# Trabalho de RL - Qual o Melhor Prompt para Iterar sobre a Geração de Código?
Nosso trabalho de RL é sobre descobrir quais os melhores prompts para iterar sobre a geração de 
código de LLMs.

## Como funciona
Criamos uma conversa com 3 participantes: Coder, Reviewer e Refiner.
Cada participante é responsável por enviar um prompt para o LLM, e inputar os resultados do LLM no
ambiente (aqui chamado de 'conversa').

### Coder
O Coder é responsável por escrever o código.  
Ele envia o prompt inicial, descrevendo o problema a ser resolvido. Definimos que todos os nossos
problemas serão de limpeza de uma base de dados csv.  
O Coder só participa da conversa 1 vez (no início) e, por isso, não o definimos como um agente RL.
Ao invés disso, ele é programado para iterar por todos os prompts n vezes, e avaliamos os resultados
das conversas com cada prompt inicial posteriormente.

### Reviewer
O Reviewer é responsável por avaliar o código gerado pelo LLM.  
Ele envia um prompt solicitando a avaliação do código gerado pelo LLM. Ele pode essa avaliação
sempre após a geração de um código que não tem nota superior à nota terminal.  
O Reviewer é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Refiner
O Refiner é responsável por refinar o código gerado pelo LLM.
Ele envia um prompt solicitando a melhoria do código gerado pelo LLM. Ele pode essa avaliação
sempre após uma revisão do Reviewer.  
O Refiner é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Prompt
Para cada participante, geramos prompts que iam de 1 a $n$ nas **escalas** das seguintes **propriedades**:
- Clareza;
- Comprimento;
- Especificidade, e
- Complexidade.

Isso totalizou até 20 prompts diferentes para cada participante. Para diminuir o espaço de ações,
optamos por usar uma estratégia mais simples:

- Para cada prompt (**comprimento da escala** x **número de propriedades**) do **Coder**;
    - Para cada **propriedade** do **Reviewer**;
        - Para cada **propriedade** do **Refiner**;
            - Geramos $m$ conversas onde:
                1. O Coder envia o prompt e adiciona o código inicial;
                2. O código é avaliado (se a nota não for terminal, prossegue);
                3. O Reviewer escolhe um dos $n$ prompts da **propriedade** e adiciona a revisão;
                4. O Refiner escolhe um dos $n$ prompts da **propriedade** e adiciona a melhoria.
                5. Se o comprimento da conversa não for terminal, volta ao passo 2.

### Avaliação do Código
O código é avaliado por um LLM usando a bibliteca `instructor`. Pedimos que o código receba uma nota
de 0 a 100 para a sua corretude e legibilidade, bem como uma curta explicação do porquê da nota 
(esse comentário é adicionado posteriormente à conversa).  
Se a nota média for superior a 95, a conversa é terminada pois consideramos que o código é bom o
suficiente.

In [4]:
# List with JSON files name
json_files_coder = [
    "json_files/prompts_clarity_coder.json",
    "json_files/prompts_size_coder.json",
    "json_files/prompts_specificity_coder.json",
    "json_files/prompts_complexity_coder.json"
]

json_files_reviewer = [
    "json_files/prompts_clarity_reviewer.json",
    "json_files/prompts_size_reviewer.json",
    "json_files/prompts_specificity_reviewer.json",
    "json_files/prompts_complexity_reviewer.json"
]

json_files_refiner = [
    "json_files/prompts_prop1_refiner.json",
    "json_files/prompts_prop2_refiner.json",
    "json_files/prompts_prop3_refiner.json",
    "json_files/prompts_prop4_refiner.json"
]

prompts_coder = []
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        prompts_coder += data

reviewer_properties = {}    
for file_name in json_files_reviewer:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in reviewer_properties:
                reviewer_properties[item["propriedade"]] = []
            reviewer_properties[item["propriedade"]].append(item['prompt'])

refiner_properties = {}
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in refiner_properties:
                refiner_properties[item["propriedade"]] = []
            refiner_properties[item["propriedade"]].append(item['prompt'])                      

In [5]:
'''
MAX_TURNS = 1
TOT_CONVERSATIONS = 2
coder = create_coder(prompts_coder)

for coder_prompt_dict in prompts_coder:
    for rev_prop, rev_prompts in reviewer_properties.items():
        for ref_prop, ref_prompts in refiner_properties.items():
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                try:
                    start_conversation(
                        coder, 
                        coder_prompt_dict, 
                        reviewer, 
                        refiner, 
                        MAX_TURNS
                    )
                except Exception as e:
                    print(f"Conversation Skipped: {e}")
            # Saving the models
            with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(reviewer, file)
            with open(f"models/refiner_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(refiner, file)
'''                

'\nMAX_TURNS = 1\nTOT_CONVERSATIONS = 2\ncoder = create_coder(prompts_coder)\n\nfor coder_prompt_dict in prompts_coder:\n    for rev_prop, rev_prompts in reviewer_properties.items():\n        for ref_prop, ref_prompts in refiner_properties.items():\n            reviewer = create_reviewer(rev_prompts)\n            refiner = create_refiner(ref_prompts)\n            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):\n                try:\n                    start_conversation(\n                        coder, \n                        coder_prompt_dict, \n                        reviewer, \n                        refiner, \n                        MAX_TURNS\n                    )\n                except Exception as e:\n                    print(f"Conversation Skipped: {e}")\n            # Saving the models\n            with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:\n                pickle.dump(reviewer, file)\n    

In [ ]:
from IPython.display import display, Markdown
from rl.code_evaluator import CodeEvaluator


MAX_TURNS = 1
TOT_CONVERSATIONS = 2
CSV_PATH = "csv_tools/data.csv"
CLEAN_CSV_PATH = "csv_tools/data_clean.csv"
coder = create_coder(prompts_coder)

for coder_prompt_dict in prompts_coder:
    for rev_prop, rev_prompts in reviewer_properties.items():
        for ref_prop, ref_prompts in refiner_properties.items():
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            evaluator = CodeEvaluator(environment=None, prompt="Evaluate the code quality", name="Code Evaluator")
            
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                #try:
                    environment = start_conversation(
                        coder, 
                        coder_prompt_dict, 
                        reviewer, 
                        refiner, 
                        MAX_TURNS,
                        csv_path=CSV_PATH,
                        clean_csv_path=CLEAN_CSV_PATH
                    )
                    for message in environment.messages:
                        display(Markdown(f"**{message['role']}**: {message['content']}"))
                
                #except Exception as e:
                #    print(f"Conversation Skipped: {e}")

            # Salva os modelos
            # with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
            #     pickle.dump(reviewer, file)
            # with open(f"models/refiner_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
            #     pickle.dump(refiner, file)
                              


Conversations:   0%|          | 0/2 [01:29<?, ?it/s]


KeyError: "I gotta clean up this messed-up CSV file...  Python, please! 🥺\n\nHere is the initial CSV content:\n                                         Series_Title  Released_Year  Runtime                        Genre  IMDB_Rating  Meta_score              Director  No_of_Votes\n0                            The Shawshank Redemption         1994.0      NaN                        Drama          NaN         NaN                   NaN          NaN\n1                                                 NaN         1972.0      NaN                 Crime, Drama          NaN         NaN                   NaN          NaN\n2                                     The Dark Knight            NaN      NaN         Action, Crime, Drama          NaN         NaN     Christopher Nolan          NaN\n3                              The Godfather: Part II            NaN      NaN                 Crime, Drama          9.0        90.0  Francis Ford Coppola          NaN\n4                                                